### Docs
* https://developers.google.com/youtube/v3/docs

In [1]:
# -*- coding: utf-8 -*-
import os, json, flask, requests, pandas as pd

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
import google.auth.transport.requests  
from googleapiclient.http import MediaFileUpload 

In [2]:
def run_me_to_refresh_the_access_token(): 
    with open('oauth2_user_credentials.json') as fr:
        credentials = google.oauth2.credentials.Credentials(**json.load(fp=fr)) 
        
    # The old access token is still vaild even you got a new valid access token. 
        
    with open('oauth2_user_credentials.json', 'w') as fw:
        credentials.refresh(google.auth.transport.requests.Request())
        creden_js = credentials.to_json()
        refreshed_credentials = json.loads(credentials.to_json())
        # we shall discard expiry before save. Try and see the consequence if you shall not.
        refreshed_credentials.pop('expiry') 
        json.dump(refreshed_credentials, fp=fw)

In [3]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
with open('oauth2_user_credentials.json') as fr:
    credentials = google.oauth2.credentials.Credentials(**json.load(fp=fr))
    
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, 
                                          API_VERSION, 
                                          credentials=credentials)

### Get channel id from username (the problem is you can hardly find one's username)
* A more practical way to find channelId is to search 'channelId' in the source code of one's youtube homepage.
  * For example, visit https://www.youtube.com/@mu_li
  * Right click the blank part of the page and search 'channelId' in the source code
  * We can find '\<meta itemprop="channelId" content="UC8WCW6C3BWLKSZ5cMzD8Gyw"\>' or '"channelId":"UC8WCW6C3BWLKSZ5cMzD8Gyw"'

In [4]:
channels = youtube.channels().list(part='id', forUsername='klauskkpm').execute()
channelId = channels['items'][0]['id']
channelId

'UCfjTOrCPnAblTngWAzpnlMA'

### List one's playlist if you know the channel id

In [5]:
channelId='UC8WCW6C3BWLKSZ5cMzD8Gyw' # Mu Li's Machine Learning Video
playlists = youtube.playlists().list(part='snippet',channelId=channelId).execute()
print([{'playlistId':i['id'], 'title':i['snippet']['title']} for i in playlists['items']])

[{'playlistId': 'PLFXJ6jwg0qW-7UM8iUTj3qKqdhbQULP5I', 'title': '论文精读'}]


### List all videos in one playlist

In [6]:
# I am going to upload my video to this playlist "PLUenxxxx_xxxxfrNm92hxxxxQdvukmViZ"
playlistId = 'PLFXJ6jwg0qW-7UM8iUTj3qKqdhbQULP5I'
videos_ids, response = [], {}
while True:
    response = youtube.playlistItems().list(
        playlistId=playlistId,
        part='snippet',
        maxResults=50,
        pageToken=response.get('nextPageToken', '')
    ).execute()  
    # Print information about each video. 
    for playlist_item in response['items']:
        title = playlist_item['snippet']['title']
        video_id = playlist_item['snippet']['resourceId']['videoId']
        videos_ids.append((video_id, title))   
    if not response.get('nextPageToken', ''): 
        break # no next page then break 

videos_ids[:3]

[('txjl_Q4jCyQ', '如何读论文'),
 ('wYmlILPsLlY', '9年后重读深度学习奠基作之一：AlexNet【论文精读】'),
 ('zjnxu8KUYKA', 'AlexNet论文逐段精读')]

In [7]:
channelId = 'UCXuqSBlHAE6Xw-yeJA0Tunw' # Linus Tech Tips
playlists, response = [], {}
while True:
    response = youtube.playlists().list(
        channelId=channelId,
        part='snippet',
        maxResults=50,
        pageToken=response.get('nextPageToken', '')
    ).execute()  
    # Print information about each video. 
    for playlist in response['items']:
        _id = playlist['id']
        title = playlist['snippet']['title']
        playlists.append((_id, title)) 
    if not response.get('nextPageToken', ''): 
        break # no next page then break 
        
playlists[:3]

[('PL8mG-RkN2uTxUd9RC2bsL1W8Ov-3wAHxv', 'AMD Ultimate Tech Upgrade'),
 ('PL8mG-RkN2uTyl1uSu_3EGkjF12eJZPfPr', 'xQc Collabs'),
 ('PL8mG-RkN2uTy8lNzJ2QeqGNXp-HBZXsdd', 'Windows Videos')]